Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [5]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
# network.add(layer(ActFunc.relu, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [6]:
X_demo = X_train.head()
X_demo

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
571,228.0,342.1,0.0,185.7,0.0,955.8,674.3,28
623,307.0,0.0,0.0,193.0,0.0,968.0,812.0,3
740,297.0,0.0,0.0,186.0,0.0,1040.0,734.0,7
750,500.0,0.0,0.0,200.0,0.0,1125.0,613.0,28
262,212.6,0.0,100.4,159.4,10.4,1003.8,903.8,56


In [7]:
y_demo = y_train.head()

In [8]:
pos = np.random.rand(31)

weights = pos[0:27]
weights_arr = []
j = 0
for i in range(len(network.layers)):
    weight_mat = np.reshape(weights[j:j + (network.layers[i].numOfNodes * 8)], (network.layers[i].numOfNodes, -1))
    weights_arr.append(weight_mat)
    j = j + (network.layers[i].numOfNodes * 8)
    input = network.layers[i].numOfNodes


bias = pos[27:31] # Total num of nodes in the NN
bias_arr = []
j = 0
for i in range(len(network.layers)):
    bias_arr.append(bias[j:j + network.layers[i].numOfNodes])
    j = j + network.layers[i].numOfNodes

In [16]:
yhat = X_demo.apply(network.forwardCalculation, args = (weights_arr, bias_arr), axis = 1)
network.sseCalculation(yhat, y_demo)

2718002.1024679583

In [10]:
class Particle:

    def __init__(self, pos, velo, alpha, beta, gamma, delta):
        self.pos = pos
        self.velo = velo
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta

    def changeVelo(self, xloc, xbest, infbest, globbest):

        # Generate rand num from beta, gamma and delta
        b = np.random.rand() * self.beta
        c = np.random.rand() * self.gamma
        d = np.random.rand() * self.delta

        self.velo = self.alpha * self.velo + b * (xbest - xloc) + c * (infbest - xloc) + d * (globbest - xloc)

In [11]:
network.layers[0].numOfNodes

3

In [ ]:
# PSO class to train weight
# The inputLen is the num of attribute of the cement: 8

class PSO:
    

    def __init__(self, network, swarmsize, alpha, beta, gamma, delta, epsilon):
        self.swarmsize = swarmsize
        self.network = network
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.particles = None
        # self.inputLen = inputLen

    def particleDim(self, inputLen):
        dim = 0
        for i in range(len(self.network.layers)):
            dim = dim + ((self.network.layers[i].numOfNodes * inputLen) + self.network.layers[i].numOfNodes) # plus 1 for bias
            inputLen = self.network.layers[i].numOfNodes
        return dim
    
    # Array of random particles
    def randParticle(self, inputLen):
        self.particles = []
        for i in range(self.swarmsize):
            np.random.seed(0) # fix the random for easier checking
            pos = np.random.rand(self.particleDim(inputLen))
            velo = np.random.rand(self.particleDim(inputLen))
            self.particles.append(Particle(pos, velo, self.alpha, self.beta, self.gamma, self.delta)) # Try to add rand particle here
        return self.particles
    
    def optimise(self, inputLen):

        # Initialize rand particle and global best
        P = self.randParticle(inputLen)
        best = []

        # Access fitness of each particle (set of weights)
        weight_index = 0
        for i in range(len(network.layers)):
        weights = pos[:weight_index]
        weights_arr = []
        j = 0
        for i in range(len(network.layers)):
            weight_mat = np.reshape(weights[j:j + (network.layers[i].numOfNodes * 8)], (network.layers[i].numOfNodes, -1))
            weights_arr.append(weight_mat)
            j = j + (network.layers[i].numOfNodes * 8)
            input = network.layers[i].numOfNodes


        bias = pos[27:31] # Total num of nodes in the NN
        bias_arr = []
        j = 0
        for i in range(len(network.layers)):
            bias_arr.append(bias[j:j + network.layers[i].numOfNodes])
            j = j + network.layers[i].numOfNodes

        # Gather information

        # Do the change velo

        # Do the change pos


    

In [13]:
swarmsize = 10
alpha = 0.5
beta = 1
gamma = 1
delta = 2
epsilon = 0.3

pso = PSO(network, swarmsize, alpha, beta, gamma, delta, epsilon)
pso.particleDim(8)




31

In [14]:
ret = pso.randParticle(8)